<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/optimum_whisper_repro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers optimum onnxruntime-gpu onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 36.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.2

In [2]:
!optimum-cli export onnx \
  --model openai/whisper-large-v2 \
  --optimize O4 \
  --device cuda \
  whisper_auto_opt_O4

2023-09-30 15:37:25.178339: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Framework not specified. Using pt to export to ONNX.
Automatic task detection to automatic-speech-recognition-with-past (possible synonyms are: speech2seq-lm-with-past).
Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.0.1+cu118
Overriding 1 configuration item(s)
	- use_cache -> False
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/modeling_whisper.py:410: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/modeling_whisper.py:

In [1]:
from optimum.pipelines import pipeline

In [2]:
auto_opt_asr_pipe = pipeline(
  "automatic-speech-recognition",
  model="whisper_auto_opt_O4/",
  accelerator="ort",
  device="cuda:0",
)

In [3]:
# not assigned correctly, fix coming very soon
feature_extractor = auto_opt_asr_pipe.tokenizer.feature_extractor
tokenizer = auto_opt_asr_pipe.tokenizer.tokenizer

auto_opt_asr_pipe.feature_extractor = feature_extractor
auto_opt_asr_pipe.tokenizer = tokenizer

In [5]:
!wget https://github.com/revdotcom/speech-datasets/raw/main/earnings22/media/4469669.mp3

--2023-09-30 16:03:27--  https://github.com/revdotcom/speech-datasets/raw/main/earnings22/media/4469669.mp3
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/revdotcom/speech-datasets/main/earnings22/media/4469669.mp3 [following]
--2023-09-30 16:03:27--  https://media.githubusercontent.com/media/revdotcom/speech-datasets/main/earnings22/media/4469669.mp3
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29628186 (28M) [audio/mpeg]
Saving to: ‘4469669.mp3’

4469669.mp3         100%[===================>]  28.25M  --.-KB/s    in 0.08s   

2023-09-30 16:03:30 (347 MB/s) - ‘4469669.mp

In [ ]:
auto_opt_asr_pipe("4469669.mp3",
     chunk_length_s=20,
     batch_size=4)["text"][:200]